In [ ]:
import zipfile

# Path to the zip file
zip_file_path = '/content/drive/MyDrive/Colab Notebooks/archive.zip'

# Path where you want to extract the contents of the zip file
extract_to_path = '/content/drive/MyDrive/Colab Notebooks/'

# Opening the zip file in read mode
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extracting all the contents into the directory specified
    zip_ref.extractall(extract_to_path)

print("Unzipping completed.")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/archive.zip'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Define paths to the train and validation directories
train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'

# Set up data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

# Prepare iterators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')  # Use 'binary' for binary classification

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

# Load the pre-trained EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Use 1 unit with sigmoid for binary classification

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model


Found 8116 images belonging to 2 classes.
Found 900 images belonging to 2 classes.
16705208/16705208 [==============================] - 2s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Save the model
model.save('my_efficientnet_model.h5')

# If you need to evaluate on a test set, you can use the validation_generator to predict
# and compare against your ground truth labels for that subset.


Epoch 1/10
253/253 [==============================] - 204s 751ms/step - loss: 0.7082 - accuracy: 0.4932 - val_loss: 0.6883 - val_accuracy: 0.5558
Epoch 2/10
253/253 [==============================] - 159s 628ms/step - loss: 0.6948 - accuracy: 0.4999 - val_loss: 0.6923 - val_accuracy: 0.5547
Epoch 3/10
253/253 [==============================] - 153s 603ms/step - loss: 0.6932 - accuracy: 0.4994 - val_loss: 0.6932 - val_accuracy: 0.4442
Epoch 4/10
 85/253 [=========>....................] - ETA: 1:41 - loss: 0.6932 - accuracy: 0.5015

KeyboardInterrupt: 

In [ ]:
# Compare different models

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet50, ResNet101, VGG16, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Set up data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

def create_model(base_model_class):
    base_model = base_model_class(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Initialize models
model_efficientnet = create_model(EfficientNetB0)
model_resnet50 = create_model(ResNet50)
model_resnet101 = create_model(ResNet101)
model_vgg16 = create_model(VGG16)
model_inceptionv3 = create_model(InceptionV3)

# Dictionary to store models
models = {
    'EfficientNetB0': model_efficientnet,
    'ResNet50': model_resnet50,
    'ResNet101': model_resnet101,
    'VGG16': model_vgg16,
    'InceptionV3': model_inceptionv3
}

# Train and evaluate models
history = {}
for name, model in models.items():
    print(f"Training {name}...")
    history[name] = model.fit(
        train_generator,
        epochs=5,
        validation_data=validation_generator)
    # Save the model
    model.save(f'{name}_model.h5')

# Compare performance
for name, hist in history.items():
    val_accuracy = max(hist.history['val_accuracy'])
    print(f"{name} Validation Accuracy: {val_accuracy:.4f}")


Found 8116 images belonging to 2 classes.
Found 900 images belonging to 2 classes.
16705208/16705208 [==============================] - 0s 0us/step


94765736/94765736 [==============================] - 0s 0us/step


171446536/171446536 [==============================] - 1s 0us/step


58889256/58889256 [==============================] - 0s 0us/step


87910968/87910968 [==============================] - 0s 0us/step


Training EfficientNetB0...
Epoch 1/5
254/254 [==============================] - 1797s 7s/step - loss: 0.7083 - accuracy: 0.5069 - val_loss: 0.6913 - val_accuracy: 0.5556
Epoch 2/5
254/254 [==============================] - 149s 585ms/step - loss: 0.6962 - accuracy: 0.5018 - val_loss: 0.6927 - val_accuracy: 0.5556
Epoch 3/5
254/254 [==============================] - 148s 582ms/step - loss: 0.6935 - accuracy: 0.4945 - val_loss: 0.6926 - val_accuracy: 0.5556
Epoch 4/5
254/254 [==============================] - 148s 583ms/step - loss: 0.6933 - accuracy: 0.5039 - val_loss: 0.6951 - val_accuracy: 0.4433
Epoch 5/5
254/254 [==============================] - 152s 599ms/step - loss: 0.6933 - accuracy: 0.4991 - val_loss: 0.6920 - val_accuracy: 0.5556


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Training ResNet50...
Epoch 1/5
254/254 [==============================] - 162s 617ms/step - loss: 0.6605 - accuracy: 0.6166 - val_loss: 0.5969 - val_accuracy: 0.7256
Epoch 2/5
254/254 [==============================] - 149s 587ms/step - loss: 0.6215 - accuracy: 0.6536 - val_loss: 0.5915 - val_accuracy: 0.7411
Epoch 3/5
254/254 [==============================] - 146s 576ms/step - loss: 0.6025 - accuracy: 0.6824 - val_loss: 0.5756 - val_accuracy: 0.7589
Epoch 4/5
254/254 [==============================] - 148s 581ms/step - loss: 0.6008 - accuracy: 0.6814 - val_loss: 0.5947 - val_accuracy: 0.6656
Epoch 5/5
254/254 [==============================] - 153s 602ms/step - loss: 0.6006 - accuracy: 0.6842 - val_loss: 0.5780 - val_accuracy: 0.6978
Training ResNet101...
Epoch 1/5
254/254 [==============================] - 169s 640ms/step - loss: 0.6481 - accuracy: 0.6450 - val_loss: 0.5699 - val_accuracy: 0.7233
Epoch 2/5
254/254 [==============================] - 156s 613ms/step - loss: 0.5937 - a

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19, ResNet152, InceptionResNetV2, DenseNet201
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Set up data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

def create_model(base_model_class, input_shape=(224, 224, 3)):
    base_model = base_model_class(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Initialize models with appropriate input shapes
model_vgg19 = create_model(VGG19, input_shape=(224, 224, 3))
model_resnet152 = create_model(ResNet152, input_shape=(224, 224, 3))
model_inceptionresnetv2 = create_model(InceptionResNetV2, input_shape=(299, 299, 3))  # InceptionResNetV2 uses 299x299 images
model_densenet201 = create_model(DenseNet201, input_shape=(224, 224, 3))

# Dictionary to store models
models = {
    'VGG19': model_vgg19,
    'ResNet152': model_resnet152,
    'InceptionResNetV2': model_inceptionresnetv2,
    'DenseNet201': model_densenet201
}

# Train and evaluate models
# Train and evaluate models with updated epoch settings
history = {}
for name, model in models.items():
    print(f"Training {name}...")
    # Set the number of epochs to 3 for all models
    num_epochs = 3

    # Adjust the image size for InceptionResNetV2
    if name == 'InceptionResNetV2':
        temp_train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(299, 299),
            batch_size=32,
            class_mode='binary')
        temp_val_generator = val_datagen.flow_from_directory(
            val_dir,
            target_size=(299, 299),
            batch_size=32,
            class_mode='binary')
        history[name] = model.fit(
            temp_train_generator,
            epochs=num_epochs,
            validation_data=temp_val_generator)
    else:
        history[name] = model.fit(
            train_generator,
            epochs=num_epochs,
            validation_data=validation_generator)

    # Save the model
    model.save(f'{name}_model.h5')

# Compare performance
for name, hist in history.items():
    val_accuracy = max(hist.history['val_accuracy'])
    print(f"{name} Validation Accuracy: {val_accuracy:.4f}")


Found 8116 images belonging to 2 classes.
Found 900 images belonging to 2 classes.
Training VGG19...
Epoch 1/3
254/254 [==============================] - 194s 760ms/step - loss: 0.6279 - accuracy: 0.6594 - val_loss: 0.5451 - val_accuracy: 0.7678
Epoch 2/3
254/254 [==============================] - 158s 622ms/step - loss: 0.5665 - accuracy: 0.7178 - val_loss: 0.5155 - val_accuracy: 0.7522
Epoch 3/3
254/254 [==============================] - 165s 649ms/step - loss: 0.5348 - accuracy: 0.7398 - val_loss: 0.4779 - val_accuracy: 0.7922
Training ResNet152...
Epoch 1/3
254/254 [==============================] - 178s 656ms/step - loss: 0.6350 - accuracy: 0.6573 - val_loss: 0.6019 - val_accuracy: 0.7344
Epoch 2/3
254/254 [==============================] - 160s 631ms/step - loss: 0.6010 - accuracy: 0.6851 - val_loss: 0.5851 - val_accuracy: 0.7256
Epoch 3/3
254/254 [==============================] - 167s 656ms/step - loss: 0.5837 - accuracy: 0.6969 - val_loss: 0.5805 - val_accuracy: 0.7456
Trainin

In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'


In [ ]:
# Test for Swin vision transformer

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import swin_t  # This is just an illustrative import; actual import may vary
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Define your transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to the size required by Swin Transformer
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize to match ImageNet's distribution
])
'''
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Flips the image horizontally with a default 50% chance
    transforms.RandomRotation(degrees=15),  # Randomly rotates the image by up to 15 degrees
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  # Randomly changes image brightness, contrast, and other properties
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])'''


val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Load the pre-trained Swin Transformer model
model = swin_t(pretrained=True)

# Modify the classifier to fit binary classification
model.head = nn.Linear(model.head.in_features, 2)

# Define loss function, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
#optimizer = AdamW(model.parameters(), lr=1e-4)
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)  # Adjust weight_decay as needed
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)  # Adjust step_size and gamma as appropriate

# Fine-tuning loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

num_epochs = 15  # Set number of epochs to 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    scheduler.step()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_predictions / total_predictions
    print(f'Epoch {epoch+1}/{num_epochs} Train loss: {epoch_loss:.4f} Accuracy: {epoch_accuracy:.4f}')

    # Validation loop
    model.eval()
    val_running_loss = 0.0
    val_correct_predictions = 0
    val_total_predictions = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_predictions += labels.size(0)

    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_epoch_accuracy = val_correct_predictions / val_total_predictions
    print(f'Epoch {epoch+1}/{num_epochs} Validation loss: {val_epoch_loss:.4f} Accuracy: {val_epoch_accuracy:.4f}')

# Save the model checkpoint
torch.save(model.state_dict(), 'swin_transformer_finetuned.pth')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Swin_T_Weights.IMAGENET1K_V1`. You can also use `weights=Swin_T_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/swin_t-704ceda3.pth" to /root/.cache/torch/hub/checkpoints/swin_t-704ceda3.pth
100%|██████████| 108M/108M [00:02<00:00, 52.0MB/s]


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import swin_t, vit_b_16  # Import Swin Transformer and ViT
from timm import create_model  # Import timm for DeiT
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Define your transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# List of models to compare
models = {
    'swin_t': swin_t(pretrained=True),
    'vit_b_16': vit_b_16(pretrained=True),
    'deit_base_patch16_224': create_model('deit_base_patch16_224', pretrained=True),
    # Add other models here following the same pattern
}

# Define training and validation functions
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_predictions / total_predictions
    return epoch_loss, epoch_accuracy

def validate(model, val_loader, criterion, device):
    model.eval()
    val_running_loss = 0.0
    val_correct_predictions = 0
    val_total_predictions = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_predictions += labels.size(0)

    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_epoch_accuracy = val_correct_predictions / val_total_predictions
    return val_epoch_loss, val_epoch_accuracy

# Training and validation process
results = {}
num_epochs = 5  # Set the number of epochs

for model_name, model in models.items():
    print(f"Training and validating model: {model_name}")

    # Modify the classifier head for each model as needed
    if model_name == 'swin_t':
        model.head = nn.Linear(model.head.in_features, 2)
    elif model_name in ['vit_b_16', 'deit_base_patch16_224']:
        # Correctly reference the classification head for ViT and DeiT models
        model.head = nn.Linear(model.head.in_features, 2)



    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
    scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

    best_val_accuracy = 0
    for epoch in range(num_epochs):
        train_loss, train_accuracy = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_accuracy = validate(model, val_loader, criterion, device)
        scheduler.step()

        print(f'Epoch {epoch+1}/{num_epochs} for {model_name} - Train loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Val loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy

    results[model_name] = best_val_accuracy
    print(f"Best validation accuracy for {model_name}: {best_val_accuracy:.4f}")

# Print comparative results
print("Comparative Analysis Results:")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy:.4f}")


Training and validating model: swin_t
Epoch 1/5 for swin_t - Train loss: 0.1819, Accuracy: 0.9199, Val loss: 0.1641, Val Accuracy: 0.9322
Epoch 2/5 for swin_t - Train loss: 0.0428, Accuracy: 0.9826, Val loss: 0.1776, Val Accuracy: 0.9544
Epoch 3/5 for swin_t - Train loss: 0.0251, Accuracy: 0.9904, Val loss: 0.1686, Val Accuracy: 0.9467
Epoch 4/5 for swin_t - Train loss: 0.0302, Accuracy: 0.9899, Val loss: 0.2700, Val Accuracy: 0.9278
Epoch 5/5 for swin_t - Train loss: 0.0335, Accuracy: 0.9880, Val loss: 0.1652, Val Accuracy: 0.9589
Best validation accuracy for swin_t: 0.9589
Training and validating model: vit_b_16


AttributeError: 'VisionTransformer' object has no attribute 'head'

TypeError: 'float' object is not iterable

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Define your transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to the size required by LeViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Load the LeViT model
model = models.levit_128s(pretrained=True)  # Example model variant

# Modify for binary classification
model.head = nn.Linear(model.head.in_features, 2)

# Loss function, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)  # Adjust as needed

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

num_epochs = 5  # Adjust as needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    scheduler.step()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_predictions / total_predictions
    print(f'Epoch {epoch+1}/{num_epochs} Train loss: {epoch_loss:.4f} Accuracy: {epoch_accuracy:.4f}')

    # Validation loop
    model.eval()
    val_running_loss = 0.0
    val_correct_predictions = 0
    val_total_predictions = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_predictions += labels.size(0)

    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_epoch_accuracy = val_correct_predictions / val_total_predictions
    print(f'Epoch {epoch+1}/{num_epochs} Validation loss: {val_epoch_loss:.4f} Accuracy: {val_epoch_accuracy:.4f}')

# Save the model
torch.save(model.state_dict(), 'levit_finetuned.pth')


AttributeError: module 'torchvision.models' has no attribute 'levit_128s'

In [ ]:
from transformers import AutoImageProcessor, LevitForImageClassification
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

image_processor = AutoImageProcessor.from_pretrained("facebook/levit-128S")
model = LevitForImageClassification.from_pretrained("facebook/levit-128S")

inputs = image_processor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for huggingface/cats-image contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/huggingface/cats-image
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `dataset

Generating test split: 0 examples [00:00, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


config.json:   0%|          | 0.00/70.0k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/32.1M [00:00<?, ?B/s]

tabby, tabby cat


In [ ]:
from transformers import LevitForImageClassification, LevitFeatureExtractor, TrainingArguments, Trainer
from datasets import load_dataset, DatasetDict, Dataset
from PIL import Image
import numpy as np
import torch
from torch.utils.data import DataLoader

# Make sure you have a GPU available for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load your custom dataset
# For this example, we'll assume your dataset is in the following format:
# /path_to_dataset/train/benign, /path_to_dataset/train/malignant, etc.

# We will create a custom dataset loader as the Huggingface datasets library does not natively support loading from a directory structure.
import os

def load_custom_dataset(path, feature_extractor):
    images = []
    labels = []
    for label, class_name in enumerate(['benign', 'malignant']):
        class_path = os.path.join(path, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            images.append(feature_extractor(Image.open(image_path).convert('RGB'), return_tensors='pt').pixel_values.squeeze())
            labels.append(label)
    return Dataset.from_dict({'image': images, 'label': labels})

# Initialize the feature extractor
feature_extractor = LevitFeatureExtractor.from_pretrained('facebook/levit-128S')

# Load the dataset
train_dataset = load_custom_dataset(train_dir, feature_extractor)
val_dataset = load_custom_dataset(val_dir, feature_extractor)

# Create a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

# Define the model
model = LevitForImageClassification.from_pretrained('facebook/levit-128S', num_labels=2)
model.to(device)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Total number of training epochs
    per_device_train_batch_size=16,  # Batch size per device during training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,     # Load the best model at the end of training
)

# Implement a custom data collator
class LevitDataCollator:
    def __call__(self, features):
        pixel_values = torch.stack([feature['image'] for feature in features])
        labels = torch.tensor([feature['label'] for feature in features])
        return {'pixel_values': pixel_values, 'labels': labels}

data_collator = LevitDataCollator()

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    data_collator=data_collator,
    tokenizer=feature_extractor,
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model('levit-fine-tuned')


In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import BeitForImageClassification

# Dataset Preparation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load and Modify BEiT Model
model = BeitForImageClassification.from_pretrained('microsoft/beit-base-patch16-224')
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 2),
    nn.Softmax(dim=1)
)

# Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training Loop
def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f"Validation Accuracy: {100 * correct / total}%")

# Start Training
train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10)

# Save the model
# torch.save(model.state_dict(), 'model.pth')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.9k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/350M [00:00<?, ?B/s]

KeyboardInterrupt: 

AttributeError: 'BreastCancerDataset' object has no attribute 'img_paths'

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


model = ViTForImageClassification.from_pretrained('google/vit-huge-patch14-224-in21k')
model.num_labels = 2
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 2),
    nn.LogSoftmax(dim=1)
)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training Loop
def train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.logits, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f"Validation Accuracy: {100 * correct / total}%")


train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, epochs=10)


torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/model.pth')


config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.53G [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-huge-patch14-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

In [ ]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 918.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Paths to your dataset

train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Load the ViT-B/16 model
model = timm.create_model("vit_base_patch16_224", pretrained=True, num_classes=3)

# Move model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# Training loop
num_epochs = 10  # Adjust epochs according to your dataset
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()

    scheduler.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}, Accuracy: {correct_predictions/len(train_loader.dataset)}')

# Evaluation loop can be added here

# Save the model
torch.save(model.state_dict(), 'vit_base_patch16_224_finetuned.pth')


KeyboardInterrupt: 

In [ ]:

train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Define the transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])




# Setup the training and validation datasets
# Make sure to replace 'your_train_path' and 'your_validation_path' with your actual dataset paths
train_dataset = datasets.ImageFolder(root='train_dir', transform=train_transform)
val_dataset = datasets.ImageFolder(root='val_dir', transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Load the ViT-L/16 model
model = timm.create_model('vit_large_patch16_224', pretrained=True, num_classes=3)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# Training and validation loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    total_train_correct = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train_correct += (predicted == labels).sum().item()

    scheduler.step()

    # Calculate training accuracy
    train_accuracy = total_train_correct / len(train_dataset)

    # Validation loop
    model.eval()
    total_val_loss = 0
    total_val_correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val_correct += (predicted == labels).sum().item()

    # Calculate validation accuracy
    val_accuracy = total_val_correct / len(val_dataset)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {total_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {total_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

# Optional: Save the trained model
torch.save(model.state_dict(), 'vit_large_patch16_224_finetuned.pth')


FileNotFoundError: [Errno 2] No such file or directory: 'train_dir'

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Paths to your dataset
train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Load the ViT-L/16 model
model_name = "vit_large_patch16_224"
model = timm.create_model(model_name, pretrained=True)
model.head = nn.Linear(model.head.in_features, 3)  # Adjust for 3 classes

# Move model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# Training loop
num_epochs = 10  # Adjust epochs according to your dataset
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()

    scheduler.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}, Accuracy: {correct_predictions/len(train_loader.dataset)}')

# Save the model
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/vit_large_patch16_224_finetuned.pth')


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Epoch 1/10, Loss: 0.4256179415119382, Accuracy: 0.8107936507936508
Epoch 2/10, Loss: 0.08306230390293576, Accuracy: 0.9678730158730159


KeyboardInterrupt: 

# **vit_large_patch16_224**

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Paths to your dataset
train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Load the ViT-L/16 model
model_name = "vit_large_patch16_224"
model = timm.create_model(model_name, pretrained=True, num_classes=3) # Make sure to set num_classes=3 for your task

# Move model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# Training and validation loop
num_epochs = 10  # Adjust epochs according to your dataset
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0

    # Training
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()

    scheduler.step()
    train_accuracy = correct_predictions / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / len(val_loader.dataset)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {total_loss/len(train_loader)}, Train Accuracy: {train_accuracy}, Val Accuracy: {val_accuracy}')

# Save the model
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/vit_large_patch16_224_finetuned.pth')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


KeyboardInterrupt: 

# **DEiT model pretrained**

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Define transformations for the training and validation datasets
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Paths to your dataset
train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

# Create data loaders with multiple workers
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, num_workers=4)

# Load the DEiT model pretrained on ImageNet
model_name = "deit_base_patch16_224"  # DEiT base model with patch size 16x16 for 224x224 input
model = timm.create_model(model_name, pretrained=True, num_classes=3)  # Adjust num_classes for your dataset

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# Training loop
num_epochs = 10  # Adjust the number of epochs according to your needs
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    correct_train_predictions = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train_predictions += (predicted == labels).sum().item()

    train_accuracy = correct_train_predictions / len(train_loader.dataset)

    # Validation phase
    model.eval()
    total_val_loss = 0
    correct_val_predictions = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_val_predictions += (predicted == labels).sum().item()

    val_accuracy = correct_val_predictions / len(val_loader.dataset)

    scheduler.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {total_train_loss/len(train_loader)}, '
          f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {total_val_loss/len(val_loader)}, '
          f'Val Accuracy: {val_accuracy:.4f}')

# Save the trained model
model_save_path = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/deit_base_patch16_224_finetuned.pth'
torch.save(model.state_dict(), model_save_path)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Paths to your dataset
train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Load the ViT-L/16 model
model_name = "vit_large_patch16_224"
model = timm.create_model(model_name, pretrained=True, num_classes=3) # Make sure to set num_classes=3 for your task

# Move model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# Training and validation loop
num_epochs = 10  # Adjust epochs according to your dataset
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0

    # Training
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()

    scheduler.step()
    train_accuracy = correct_predictions / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / len(val_loader.dataset)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {total_loss/len(train_loader)}, Train Accuracy: {train_accuracy}, Val Accuracy: {val_accuracy}')

# Save the model
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/vit_large_patch16_224_finetuned.pth')


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Epoch 1/10, Train Loss: 0.25908394926981737, Train Accuracy: 0.9097142857142857, Val Accuracy: 0.9644444444444444
Epoch 2/10, Train Loss: 0.08836297818830968, Train Accuracy: 0.9678730158730159, Val Accuracy: 0.9588888888888889
Epoch 3/10, Train Loss: 0.043296199162177654, Train Accuracy: 0.9837460317460317, Val Accuracy: 0.9622222222222222
Epoch 4/10, Train Loss: 0.034900929759129465, Train Accuracy: 0.9876825396825397, Val Accuracy: 0.9333333333333333
Epoch 5/10, Train Loss: 0.035190465332156834, Train Accuracy: 0.9876825396825397, Val Accuracy: 0.9444444444444444
Epoch 6/10, Train Loss: 0.04504864341321847, Train Accuracy: 0.984, Val Accuracy: 0.97
Epoch 7/10, Train Loss: 0.024340757173293374, Train Accuracy: 0.9906031746031746, Val Accuracy: 0.9611111111111111
Epoch 8/10, Train Loss: 0.00796421914886008, Train Accuracy: 0.9961904761904762, Val Accuracy: 0.9666666666666667
Epoch 9/10, Train Loss: 0.0073369220106925285, Train Accuracy: 0.9960634920634921, Val Accuracy: 0.965555555555

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from torch.cuda.amp import GradScaler, autocast  # for mixed precision training

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Paths to your dataset
train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

# Adjust the batch size and num_workers to your environment
batch_size = 64  # Adjust the batch size based on your GPU memory
num_workers = 8  # Increase the number of workers to improve data loading

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers)

# Load the DINOV2 model (make sure the model name is correct)
model_name = "vit_small_patch16_224.dino"  # Update this to the correct model name if using DINOv2
model = timm.create_model(model_name, pretrained=True, num_classes=3)

# Move model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)
scaler = GradScaler()  # for mixed precision training

# Training and validation loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss, correct_predictions = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Enables autocasting for the forward pass (model + loss)
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        # Scales loss, calls backward to create scaled gradients
        optimizer.zero_grad()
        scaler.scale(loss).backward()

        # Unscales the gradients of optimizer's assigned params in-place
        scaler.step(optimizer)
        # Updates the scale for next iteration
        scaler.update()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()

    scheduler.step()
    train_accuracy = correct_predictions / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / len(val_loader.dataset)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {total_loss/len(train_loader)}, Train Accuracy: {train_accuracy}, Val Accuracy: {val_accuracy}')

# Save the model
model_save_path = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/vit_large_patch16_224'


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1/10, Train Loss: 0.849868930876255, Train Accuracy: 0.6462222222222223, Val Accuracy: 0.7511111111111111
Epoch 2/10, Train Loss: 0.5229712489151186, Train Accuracy: 0.7357460317460317, Val Accuracy: 0.7466666666666667
Epoch 3/10, Train Loss: 0.4392589609469137, Train Accuracy: 0.7958095238095239, Val Accuracy: 0.7866666666666666
Epoch 4/10, Train Loss: 0.37917192881145784, Train Accuracy: 0.8307301587301588, Val Accuracy: 0.8133333333333334
Epoch 5/10, Train Loss: 0.34504530218339735, Train Accuracy: 0.8467301587301588, Val Accuracy: 0.6011111111111112
Epoch 6/10, Train Loss: 0.350755161906202, Train Accuracy: 0.8372063492063492, Val Accuracy: 0.7988888888888889
Epoch 7/10, Train Loss: 0.25537692839580195, Train Accuracy: 0.8933333333333333, Val Accuracy: 0.7444444444444445
Epoch 8/10, Train Loss: 0.17919889077424042, Train Accuracy: 0.9315555555555556, Val Accuracy: 0.8866666666666667
Epoch 9/10, Train Loss: 0.11579424579207215, Train Accuracy: 0.9565714285714285, Val Accuracy:

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from torchvision.models import resnet50

def train_and_validate(model_name, train_loader, val_loader, device, num_classes=3, num_epochs=10):
    # Load the model
    if model_name == "resnet50":
        model = resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
    model.to(device)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

    # Training and validation loop
    for epoch in range(num_epochs):
        model.train()
        total_train_loss, correct_train_predictions = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train_predictions += (predicted == labels).sum().item()

        scheduler.step()
        train_accuracy = correct_train_predictions / len(train_loader.dataset)

        # Validation phase
        model.eval()
        total_val_loss, correct_val_predictions = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct_val_predictions += (predicted == labels).sum().item()

        val_accuracy = correct_val_predictions / len(val_loader.dataset)

        print(f'Model: {model_name}, Epoch {epoch+1}/{num_epochs}, Train Loss: {total_train_loss/len(train_loader)}, '
              f'Train Accuracy: {train_accuracy}, Val Loss: {total_val_loss/len(val_loader)}, Val Accuracy: {val_accuracy}')

    # Save the model
    torch.save(model.state_dict(), f'{model_name}_finetuned.pth')

# Define transformations
# ... (same as before)

# Paths to your dataset
# ... (same as before)

# Load datasets
# ... (same as before)

# Data loaders
# ... (same as before)

# List of models to train and validate
model_names = [
    "vit_large_patch16_224",
    "vit_small_patch16_224_dino",  # Adjust the name if needed based on the timm library
    "deit_base_patch16_224",
    "swin_tiny_patch4_window7_224",  # Example of a SWIN Transformer
    "resnet50"  # A non-transformer model for comparison
]

# Train and validate each model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for model_name in model_names:
    print(f"Training and validating model {model_name}")
    train_and_validate(model_name, train_loader, val_loader, device)
